In [43]:
import pandas as pd
import numpy as np
import sys

import matplotlib.pyplot as plt
# from bertopic import BERTopic
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
from scipy import stats

In [51]:
news = pd.read_csv('results/news.csv')
news_tweets = pd.read_csv('results/news_tweets.csv')
user_resp = pd.read_csv('results/user_resp.csv')

/tmp/ipykernel_2535612/4151079534.py:3: DtypeWarning: Columns (0,7,8,9,14,15,16,18,35,36,38,45,46,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  user_resp = pd.read_csv('results/user_resp.csv')


In [52]:
user_resp.shape

(4039629, 61)

In [778]:
news.topics.value_counts()

other           7457
covid           5265
election        4986
social_media    2457
internatinal    2392
democrats       2039
capitol         1577
floyd           1386
economy         1305
court           1277
security         909
republican       870
immigration      817
hunter_biden     737
pres_debate      710
climate          655
first_lady       365
abortion         255
healthcare       215
proud boys       212
Name: topics, dtype: int64

In [779]:
news_tweets.topics.value_counts()

election             4680
covid                3161
other                2952
none                 2901
democrats            1561
national             1320
republican           1119
economy               949
media                 886
blm                   769
supreme_court         737
white_house           668
conspiracy_theory     659
american              574
newyork               417
international         365
capitol               336
impeachment           261
healthcare            165
immigration           104
Name: topics, dtype: int64

In [557]:
R_news_tweets[((R_news_tweets.Biden_flag == True) & (R_news_tweets.topics == 'healthcare'))].shape

(9, 49)

In [5]:
news_tweets.shape, news_tweets_subset.shape

((24584, 49), (24584, 12))

In [4]:
news_tweets_subset = news_tweets[['conversation_id', 'topics', 'subtopic', 'trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new', 'Trump_flag', 'Biden_flag', 'Trump_Biden_flag']]

In [6]:
# user_resp = user_resp.merge(news_tweets_subset, on = 'conversation_id')
user_resp = user_resp.rename(columns={"Trump_flag": "trump_flag", "Biden_flag": "biden_flag"})

In [7]:
user_resp.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'tweet_id', 'conversation_id',
       'author_id', 'created_at', 'geo', 'lang', 'like_count', 'quote_count',
       'reply_count', 'retweet_count', 'source', 'text', 'anger', 'joy',
       'optimism', 'sadness', 'publication', 'topic_labels', 'theme',
       'pos_senti', 'neu_senti', 'neg_senti', 'date', 'week', 'flag',
       'trump_flag', 'biden_flag', 'Trump_Biden_flag', 'trump_pos',
       'trump_neg', 'trump_neu', 'biden_pos', 'biden_neg', 'biden_neu',
       'topics', 'topic_ids', 'subtopic'],
      dtype='object')

In [53]:
R_user_resp = user_resp[((user_resp['publication'] == 'Breitbart News') | (user_resp['publication'] == 'Fox News'))]
L_user_resp = user_resp[((user_resp['publication'] == 'CNN') | (user_resp['publication'] == 'The Washington Post'))]
C_user_resp = user_resp[((user_resp['publication'] == 'Business Insider') | (user_resp['publication'] ==  'USA Today'))]

In [9]:
R_news_tweets = news_tweets[((news_tweets['publication'] == 'Breitbart News') | (news_tweets['publication'] == 'Fox News'))]
L_news_tweets = news_tweets[((news_tweets['publication'] == 'CNN') | (news_tweets['publication'] == 'The Washington Post'))]
C_news_tweets = news_tweets[((news_tweets['publication'] == 'Business Insider') | (news_tweets['publication'] ==  'USA Today'))]

In [54]:
senti = ['neg', 'neu', 'pos']
emotions = ['anger', 'sadness', 'joy', 'optimism']

In [55]:
from pingouin import kruskal

def get_KW_stat(emot1, emot2):
    
    df1 = pd.DataFrame(emot1.values, columns = ['senti'])
    df1['type_flag'] = 1

    df2 = pd.DataFrame(emot2.values, columns = ['senti'])
    df2['type_flag'] = 2
    
    temp_df = pd.concat((df1,df2), axis = 0)
    kwTest = kruskal(temp_df, dv='senti', between='type_flag')
    H = kwTest.H[0]
    p = kwTest['p-unc'][0]
    dof = kwTest['ddof1'][0]
    n = temp_df.shape[0]
    esq = H * (n + 1)/(n**2 - 1)
    return H, p, dof, esq


def get_pos_neg_mentions(df, entity):
    
    entity_mentions = df[df[entity + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos_new'] > pos_mean))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg_new'] > neg_mean))]
    
    return pos_entity_mentions, neg_entity_mentions

def get_significance_stats(dist1, dist2, pub, entity, emotion):

    emot1_shapiro = stats.shapiro(dist1)
    emot2_shapiro = stats.shapiro(dist2)
    emot1_shapiro = (emot1_shapiro.statistic, emot1_shapiro.pvalue)
    emot2_shapiro = (emot2_shapiro.statistic, emot2_shapiro.pvalue)
    H, p, dof, es = get_KW_stat(dist1, dist2)
    # t, p , dof = welch_ttest(emot1, emot2)
    res = [entity, pub, emotion, dist1.shape[0], dist2.shape[0], emot1_shapiro[0], emot1_shapiro[1], emot2_shapiro[0], emot2_shapiro[1], p, H, dof, es]
    # print(res)
    return res

def get_mean_emotions(df):
    
    return df['anger'].mean(), df['joy'].mean(), df['optimism'].mean(), df['sadness'].mean()

def get_mean_emotions_per_sentiment(df, entity):
    
    entity_mentions = df[df[entity + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos_new'].mean()
    neu_mean = entity_mentions[entity + '_neu_new'].mean()
    neg_mean = entity_mentions[entity + '_neg_new'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos_new'] > pos_mean * 1.5))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg_new'] > neg_mean * 1.5))]
    
#     print(entity, pos_entity_mentions.shape, pos_entity_mentions_.shape)
#     print(entity, neg_entity_mentions.shape, neg_entity_mentions_.shape)
    
    pos_tweet_emot = get_mean_emotions(pos_entity_mentions)
    neg_tweet_emot = get_mean_emotions(neg_entity_mentions)
    
    return pos_entity_mentions, neg_entity_mentions, pos_tweet_emot, neg_tweet_emot

### News headlines statistical test

In [110]:
news.columns

Index(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'id', 'author', 'date',
       'title', 'publication', 'trump_neg', 'trump_neu', 'trump_pos',
       'biden_neg', 'biden_neu', 'biden_pos', 'trump_flag', 'biden_flag',
       'vad_neg', 'vad_neu', 'vad_pos', 'vad_comp', 'text', 'processed_text',
       'topic_ids', 'topics', 'subtopic', 'trump_pos_new', 'trump_neg_new',
       'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new'],
      dtype='object')

In [16]:
R_news = news[((news['publication'] == 'Breitbart News') | (news['publication'] == 'Fox News'))]
L_news = news[((news['publication'] == 'CNN') | (news['publication'] == 'The Washington Post'))]
C_news = news[((news['publication'] == 'Business Insider') | (news['publication'] ==  'USA Today'))]

In [17]:
R_news.shape, L_news.shape, C_news.shape

((15704, 32), (11163, 32), (9019, 32))

In [40]:
news_tweets[news_tweets.conversation_id == 1330876016131248128].topics

11633    conspiracy_theory
Name: topics, dtype: object

In [41]:
# user_resp[user_resp.topics == 'fact-check'][['tweet_id', 'conversation_id']].astype(int)

In [ ]:
cols = ['trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new']

In [238]:
R_news[cols].describe()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,id,trump_neg,trump_neu,trump_pos,biden_neg,biden_neu,...,vad_neu,vad_pos,vad_comp,topic_ids,trump_pos_new,trump_neg_new,trump_neu_new,biden_pos_new,biden_neg_new,biden_neu_new
count,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,15704.000000,...,15704.000000,15704.000000,15704.000000,15704.000000,10888.000000,10888.000000,10888.000000,6692.000000,6692.000000,6692.000000
mean,17618.150662,17618.150662,21824.213449,21824.213449,75807.339277,0.243534,0.225642,0.074574,0.090231,0.175151,...,0.806998,0.084255,-0.051399,20.203324,0.161783,0.503286,0.334931,0.177913,0.409237,0.412850
std,10108.822932,10108.822932,12329.709858,12329.709858,40083.431340,0.389467,0.358965,0.220057,0.244702,0.303643,...,0.163541,0.121035,0.395391,20.006580,0.259269,0.392599,0.328652,0.255087,0.372870,0.326753
min,0.000000,0.000000,1.000000,1.000000,59.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.201000,0.000000,-0.946800,0.000000,0.001367,0.004249,0.004203,0.001306,0.004645,0.004550
25%,8995.750000,8995.750000,11459.750000,11459.750000,42183.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.693000,0.000000,-0.361200,4.000000,0.007307,0.082021,0.045430,0.014483,0.063505,0.094535
50%,17696.500000,17696.500000,21936.500000,21936.500000,78171.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.806000,0.000000,0.000000,13.000000,0.027474,0.485243,0.187865,0.046851,0.248991,0.346453
75%,25711.250000,25711.250000,31669.500000,31669.500000,107331.500000,0.629032,0.570691,0.000000,0.000000,0.460205,...,1.000000,0.169000,0.202300,31.000000,0.173959,0.936599,0.628287,0.228202,0.832834,0.731991
max,35800.000000,35800.000000,43723.000000,43723.000000,144521.000000,0.992451,0.983506,0.956413,0.987192,0.971152,...,1.000000,0.739000,0.959500,77.000000,0.974883,0.994074,0.980883,0.975993,0.992216,0.977706


In [111]:
news['trump_flag'] = news.title.str.contains('Trump')
news['biden_flag'] = news.title.str.contains('Biden')

### Comparing sentiment distributions (toward an entity) across L and R news outlets 

In [505]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
sstat = []
for ent in ['biden', 'trump']:
    
    # l_df_pos, l_df_neg, _, _ = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent + '_flag'] == True], ent)
    # r_df_pos, r_df_neg, _, _ = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent + '_flag'] == True], ent)

    for senti in ['neg', 'pos']:
        
        # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
        dist1 = L_news_tweets[L_news_tweets[ent.title() + '_flag'] == True][ent + '_' + senti + '_new'].dropna()
        dist2 = R_news_tweets[R_news_tweets[ent.title() + '_flag'] == True][ent + '_' + senti + '_new'].dropna()
        
#         n = min(dist1.shape[0], dist2.shape[0])
        
#         dist1 = dist1.sample(n)
#         dist2 = dist2.sample(n)
        
        # print(stats.shapiro(dist1), stats.shapiro(dist2))
        # sys.exit()

        res = get_significance_stats(dist1, dist2, ent, senti, 'None')
        # print(res)
        sstat.append(res)
        # sys.exit()
    # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,neg,biden,None,3687,1933,0.723,0.0,0.836,0.0,0.0,437.627,1,0.078
1,pos,biden,None,3687,1933,0.864,0.0,0.720,0.0,0.0,508.343,1,0.090
2,neg,trump,None,10645,2656,0.811,0.0,0.792,0.0,0.0,482.047,1,0.036
3,pos,trump,None,10645,2656,0.618,0.0,0.703,0.0,0.0,164.748,1,0.012


In [506]:
stats_df.to_csv('results/news_headline_analysis/statistical_test_L_vs_R_news_tweets.csv')

In [12]:
news.topics.unique()

array(['covid', 'social_media', 'election', 'other', 'economy',
       'internatinal', 'court', 'democrats', 'healthcare', 'climate',
       'republican', 'security', 'immigration', 'floyd', 'capitol',
       'hunter_biden', 'proud boys', 'first_lady', 'abortion',
       'pres_debate'], dtype=object)

### Comparing sentiment distributions separately for different topics

In [49]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_news_tweets.topics).intersection(set(R_news_tweets.topics))
sstat = []
for ent in ['biden', 'trump']:
    
    # l_df_pos, l_df_neg, _, _ = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent + '_flag'] == True], ent)
    # r_df_pos, r_df_neg, _, _ = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent + '_flag'] == True], ent)
    
    for top in topics:
        
        dataL = L_news_tweets[L_news_tweets.topics == top]
        dataR = R_news_tweets[R_news_tweets.topics == top]

        for senti in ['neg', 'pos']:

            # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
            dist1 = dataL[dataL[ent.title() + '_flag'] == True][ent + '_' + senti + '_new'].dropna()
            dist2 = dataR[dataR[ent.title() + '_flag'] == True][ent + '_' + senti + '_new'].dropna()
            
#             n = min(dist1.shape[0], dist2.shape[0])
        
#             dist1 = dist1.sample(n)
#             dist2 = dist2.sample(n)
            # print(dist1.shape, dist2.shape)

            # print(stats.shapiro(dist1), stats.shapiro(dist2))
            # sys.exit()
            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                res = get_significance_stats(dist1, dist2, ent, senti, top)
                # print(res)
                sstat.append(res)
                
            else:
                print(top, senti, dist1.shape, dist2.shape)
            # sys.exit()
        # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

capitol neg (24,) (8,)
capitol pos (24,) (8,)
healthcare neg (17,) (9,)
healthcare pos (17,) (9,)
healthcare neg (98,) (9,)
healthcare pos (98,) (9,)


,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,neg,biden,newyork,29,48,0.734,0.000,0.804,0.000,0.001,11.890,1,0.156
1,pos,biden,newyork,29,48,0.871,0.002,0.653,0.000,0.000,15.130,1,0.199
2,neg,biden,white_house,65,35,0.596,0.000,0.743,0.000,0.028,4.842,1,0.049
3,pos,biden,white_house,65,35,0.874,0.000,0.707,0.000,0.001,10.319,1,0.104
4,neg,biden,american,84,60,0.724,0.000,0.785,0.000,0.001,11.041,1,0.077
5,pos,biden,american,84,60,0.879,0.000,0.771,0.000,0.000,13.449,1,0.094
6,neg,biden,election,975,358,0.730,0.000,0.825,0.000,0.000,59.660,1,0.045
7,pos,biden,election,975,358,0.877,0.000,0.739,0.000,0.000,109.458,1,0.082
8,neg,biden,none,451,177,0.727,0.000,0.827,0.000,0.000,35.662,1,0.057
9,pos,biden,none,451,177,0.879,0.000,0.685,0.000,0.000,53.870,1,0.086


In [515]:
# dist1 = R_news[R_news.topics == 'security']['trump_pos_new'].sample(200)
# dist2 = R_news[R_news.topics == 'security']['trump_pos_new'].sample(200)

# get_significance_stats(dist1, dist2, 'Trump', senti, top)

In [516]:
 # L_news[L_news.topics == 'security']['trump_pos_new'].sample(200)

In [523]:
stats_df[['senti', 'Entity', 'Topic', 'KW-p', 'KW-H', 'ES']].to_csv('results/statistical_analysis/statistical_test_L_vs_R_news_topic_wise.csv')

In [528]:
stats_df.sort_values(by=['ES'], ascending=False) #.iloc[0:25].Entity.value_counts()

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
35,pos,biden,newyork,29,48,0.871,0.002,0.653,0.000,0.000,15.130,1,0.199
22,neg,biden,impeachment,30,22,0.681,0.000,0.816,0.001,0.005,8.031,1,0.157
34,neg,biden,newyork,29,48,0.734,0.000,0.804,0.000,0.001,11.890,1,0.156
12,neg,biden,democrats,511,294,0.634,0.000,0.836,0.000,0.000,123.307,1,0.153
26,neg,biden,blm,100,90,0.748,0.000,0.861,0.000,0.000,27.728,1,0.147
23,pos,biden,impeachment,30,22,0.861,0.001,0.726,0.000,0.007,7.313,1,0.143
1,pos,biden,republican,117,60,0.843,0.000,0.515,0.000,0.000,24.095,1,0.137
0,neg,biden,republican,117,60,0.724,0.000,0.861,0.000,0.000,21.608,1,0.123
13,pos,biden,democrats,511,294,0.883,0.000,0.758,0.000,0.000,96.729,1,0.120
3,pos,biden,economy,103,60,0.893,0.000,0.743,0.000,0.000,19.247,1,0.119


In [208]:
news[((news.subtopic == 'republican-first_lady') & (news.title.str.contains('Biden') == True))][['title', 'date']].shape

(68, 2)

In [210]:
news[((news.subtopic == 'republican-first_lady') & (news.title.str.contains('Trump') == True))][['title', 'date']]

,title,date
1020,Inside former White House press secretary Stephanie Grisham's rocky rise to the top of the Trump administration,2020-04-07 23:35:14
1389,Donald Trump Marks Separated Easter Sunday: Were Getting Rid of the Plague,2020-04-12 17:16:38
1396,"Photos: Melania Trump Wishes Happy Easter in Kelly Green Shirt Dress, Snakeskin Pumps",2020-04-12 00:29:36
1426,Watch: Melania Trump Delivers Message of Hope to Americas Children,2020-04-13 23:52:46
1525,"In lieu of White House Easter Egg Roll, Melania Trump shares video reading Easter-themed book",2020-04-13 17:49:04
2010,Melania Trump picks up telephone diplomacy during coronavirus crisis,2020-04-18 14:56:09
2342,Trump plants WH maple tree; reopens national parks,2020-04-22 09:00:09
2351,Photos: Melania Trump Plants Trees in Victoria Beckham Trench,2020-04-23 20:55:03
2464,"A look inside the daily routine of first lady Melania Trump, who eats 7 pieces of fruit a day, is a 'full-time mom', and has been self-isolating at the White House during the coronavirus pandemic",2020-04-24 10:34:18
2626,Watch: Cardinal Dolan Welcomes Donald Trump to Livestream Mass,2020-04-26 14:26:38


### Comparing sentiment toward Trump vs Biden for each news outlet

In [135]:
sstat = []
for senti in ['neg', 'pos']:

    for n, grp in news.groupby('publication'):

        # print(n, grp.shape)
        dist1 = grp[grp['trump_flag'] == True]['trump_' + senti + '_new']
        dist2 = grp[grp['biden_flag'] == True]['biden_' + senti + '_new']
        
        res = get_significance_stats(dist1, dist2, n, senti, 'None')
            # print(res)
        sstat.append(res)
        
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Pub', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate f

,senti,Pub,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,neg,Breitbart News,None,5154,2972,0.814,0.0,0.825,0.0,0.0,144.198,1,0.018
1,neg,Business Insider,None,4074,1206,0.808,0.0,0.775,0.0,0.0,689.017,1,0.131
2,neg,CNN,None,5397,1643,0.826,0.0,0.671,0.0,0.0,1099.357,1,0.156
3,neg,Fox News,None,5734,3720,0.827,0.0,0.819,0.0,0.0,88.246,1,0.009
4,neg,The Washington Post,None,3896,1198,0.820,0.0,0.751,0.0,0.0,826.089,1,0.162
5,neg,USA Today,None,3395,1352,0.826,0.0,0.692,0.0,0.0,423.079,1,0.089
6,pos,Breitbart News,None,5154,2972,0.657,0.0,0.712,0.0,0.0,115.301,1,0.014
7,pos,Business Insider,None,4074,1206,0.591,0.0,0.832,0.0,0.0,652.408,1,0.124
8,pos,CNN,None,5397,1643,0.544,0.0,0.784,0.0,0.0,842.183,1,0.120
9,pos,Fox News,None,5734,3720,0.647,0.0,0.691,0.0,0.0,78.014,1,0.008


In [561]:
L_user_resp.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'tweet_id', 'conversation_id',
       'author_id', 'created_at', 'geo', 'lang', 'like_count', 'quote_count',
       'reply_count', 'retweet_count', 'source', 'text', 'anger', 'joy',
       'optimism', 'sadness', 'publication', 'topic_labels', 'theme',
       'pos_senti', 'neu_senti', 'neg_senti', 'date', 'week', 'flag',
       'Trump_flag', 'Biden_flag', 'Trump_Biden_flag', 'trump_pos',
       'trump_neg', 'trump_neu', 'biden_pos', 'biden_neg', 'biden_neu',
       'topics', 'topic_ids', 'subtopic'],
      dtype='object')

### User response Statistical test

### Left Vs Right (emotion distribution) across two entities (Trump or Biden)

In [57]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
sstat = []

def get_mean_emotions(df):
    
    return df['anger'].mean(), df['joy'].mean(), df['optimism'].mean(), df['sadness'].mean()


for ent in ['biden', 'trump']:
    # temp_df = user_resp[user_resp['publication'] == pub]
    # for ent in ['biden', 'trump']:
    
    l_df = L_user_resp[L_user_resp[ent.title() + '_flag'] == True]
    r_df = R_user_resp[R_user_resp[ent.title() + '_flag'] == True]


    for emot in emotions:
    # for emot in ['neg_senti', 'neu_senti', 'pos_senti']:

        dist1 = l_df[emot].dropna()
        dist2 = r_df[emot].dropna()
        if(len(dist1) > 10 and len(dist2) < 10):
            res = get_significance_ss(dist1, dist2, ent, senti, emot)
        else:
            res = None
        # print(res)
        sstat.append(res)
        # sys.exit()
    # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

ValueError: Shape of passed values is (8, 1), indices imply (8, 13)

### Left Vs Right (emotion distribution) across two entities (Trump or Biden) based on positive and negative mentions

In [48]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
sstat = []

def get_mean_emotions(df):
    
    return df['anger'].mean(), df['joy'].mean(), df['optimism'].mean(), df['sadness'].mean()


def get_mean_emotions_per_sentiment(df, entity):
    
    entity_mentions = df[df[entity.title() + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos'] > pos_mean * 1.5))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg'] > neg_mean * 1.5))]
    
#     print(entity, pos_entity_mentions.shape, pos_entity_mentions_.shape)
#     print(entity, neg_entity_mentions.shape, neg_entity_mentions_.shape)
    
    pos_tweet_emot = get_mean_emotions(pos_entity_mentions)
    neg_tweet_emot = get_mean_emotions(neg_entity_mentions)
    
    return pos_entity_mentions, neg_entity_mentions, pos_tweet_emot, neg_tweet_emot


for ent in ['biden', 'trump']:
    # temp_df = user_resp[user_resp['publication'] == pub]
    # for ent in ['biden', 'trump']:
    
    l_df_pos, l_df_neg, _, _ = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent.title() + '_flag'] == True], ent)
    r_df_pos, r_df_neg, _, _ = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent.title() + '_flag'] == True], ent)

    for senti in ['Mneg', 'Mpos']:
        
        if(senti == 'Mneg'):
            # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
            left_df, right_df = l_df_neg, r_df_neg
            # print(l_df_neg.sample(n).shape)
            # print(r_df_neg.sample(n).shape)
            # sys.exit()
        else:
            # n = min(l_df_pos.shape[0], r_df_pos.shape[0])
            left_df, right_df = l_df_pos, r_df_pos
            
        for emot in emotions:
        # for emot in ['neg_senti', 'neu_senti', 'pos_senti']:

            dist1 = left_df[emot].dropna()
            dist2 = right_df[emot].dropna()
            res = get_significance_stats(dist1, dist2, ent, senti, emot)
            # print(res)
            sstat.append(res)
            # sys.exit()
    # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/home/ahaque2/venv/py3_10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate f

,senti,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,Mneg,biden,anger,45801,9156,0.811,0.0,0.777,0.0,0.000,68.818,1,0.001
1,Mneg,biden,joy,45801,9156,0.480,0.0,0.490,0.0,0.765,0.089,1,0.000
2,Mneg,biden,optimism,45801,9156,0.624,0.0,0.572,0.0,0.000,244.736,1,0.004
3,Mneg,biden,sadness,45801,9156,0.620,0.0,0.582,0.0,0.000,46.569,1,0.001
4,Mpos,biden,anger,45309,11900,0.810,0.0,0.803,0.0,0.000,21.484,1,0.000
5,Mpos,biden,joy,45309,11900,0.521,0.0,0.494,0.0,0.006,7.535,1,0.000
6,Mpos,biden,optimism,45309,11900,0.622,0.0,0.617,0.0,0.000,21.841,1,0.000
7,Mpos,biden,sadness,45309,11900,0.609,0.0,0.588,0.0,0.000,14.411,1,0.000
8,Mneg,trump,anger,184492,29023,0.803,0.0,0.772,0.0,0.000,418.729,1,0.002
9,Mneg,trump,joy,184492,29023,0.500,0.0,0.515,0.0,0.000,36.497,1,0.000


In [140]:
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
len(topics)

18

### Left vs Right (emotion distributions) based on topics

In [1]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
sstat = []
for emot in emotions:
    
    for ent in ['biden', 'trump']:
        
        dataL = L_user_resp[L_user_resp[ent.title() + '_flag'] == True]
        dataR = R_user_resp[R_user_resp[ent.title() + '_flag'] == True]
        
        for top in topics:
            
            dist1 = dataL[dataL.topics == top][emot].dropna()
            dist2 = dataR[dataR.topics == top][emot].dropna()
            
            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                res = get_significance_stats(dist1, dist2, ent, top, emot)
                # print(res)
                sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

NameError: name 'L_user_resp' is not defined

In [583]:
stats_df[stats_df['ES'] > 0.05]

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
7,media,biden,anger,3280,4374,0.730,0.0,0.846,0.0,0.0,517.314,1,0.068
43,media,biden,joy,3280,4374,0.453,0.0,0.601,0.0,0.0,482.836,1,0.063


In [188]:
pd.set_option('display.max_rows', None)
stats_df.sort_values(by=['ES'], ascending=False).iloc[0:25].Entity.value_counts()

biden    21
trump     4
Name: Entity, dtype: int64

In [182]:
news_tweets[((news_tweets.subtopic == 'democarts-general') & (news_tweets.text.str.contains('Biden') == True))][['text', 'date']]

,text,date
160,"House Democrats are working on drafting legislation proposed by President Biden that would direct the IRS to send recurring monthly payments to American families to expand the child tax credit, a source familiar with the matter confirmed to CNN. https://t.co/SjusEAY96l",2021-01-23 21:45:06+00:00
1770,"Joe Biden is projected to win an election that another Democrat, especially a more liberal one, may very well have lost | Analysis by CNN's Harry Enten https://t.co/zF5Sh4ON60",2020-11-08 23:00:05+00:00
1804,"BREAKING: JOE BIDEN WINS\n\nJoe Biden will be the 46th president of the United States, CNN projects, after a victory in Pennsylvania puts the Scranton-born Democrat over 270 https://t.co/qmVt37WlYC #CNNElection https://t.co/Ts97NweP99",2020-11-07 16:40:59+00:00
1805,"BREAKING: JOE BIDEN WINS\n\nJoe Biden will be the 46th president of the United States, CNN projects, after a victory in Pennsylvania puts the Scranton-born Democrat over 270 https://t.co/g5ahxZ3Zcu #CNNElection https://t.co/4bVHYENaaT",2020-11-07 16:33:00+00:00
1806,"BREAKING: JOE BIDEN WINS\n\nJoe Biden will be the 46th president of the United States, CNN projects, after a victory in Pennsylvania puts the Scranton-born Democrat over 270 https://t.co/pZ8pr4cMTG #CNNElection https://t.co/4pbJtzY1FT",2020-11-07 16:27:38+00:00
3025,"""The reality of Joe Biden's policy proposals stands in sharp contrast to a campaign that has positioned him as a centrist Democrat,"" Lanhee Chen for @CNNOpinion. ""His platform reflects the triumph of the progressive left."" https://t.co/cVeGDnrzdF",2020-09-21 14:40:03+00:00
3492,"Here’s why Democrats are worried about Kenosha, @ForecasterEnten tells @smerconish.\n “Numbers right now still indicate that Joe Biden is ahead, but…the events over the last two months have not exactly been helpful to the former vice president's campaign.” https://t.co/KNzvbK4zFV https://t.co/uubh7ld6sf",2020-08-29 13:22:05+00:00
3542,"Increases in the use of positives when discussing Biden came across the board, but the biggest jumps came from Democrats, women, Latinos and college graduates. https://t.co/4Hv3JFVdab",2020-08-27 17:40:05+00:00
3597,"""Black Americans like me are free people with free minds. You hear that Joe Biden? We are not a monolithic thinking group subservient to Democrats' electoral wishes,"" writes Georgia state Rep. Vernon Jones, a Democrat who spoke at #RNC2020, for @CNNOpinion https://t.co/2YLePaBZ64",2020-08-25 22:27:01+00:00
3708,"Sen. Chris Coons said he will highlight the impact of faith in Joe Biden’s life when he introduces him at the #DemConvention tonight. “For too long, Democrats have been retiring, shy, quiet about our faith and about the ways in which our faith inspires us to act.” https://t.co/ptd0OSgvbI",2020-08-21 01:22:05+00:00


In [170]:
news_tweets.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'tweet_id', 'conversation_id', 'author_id', 'created_at', 'geo', 'lang',
       'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source',
       'text', 'display_name', 'topics', 'topic_labels', 'theme', 'pos_senti',
       'neu_senti', 'neg_senti', 'publication', 'date', 'week', 'polIncl',
       'hashtag', 'clean_text', 'proc_text', 'Segmented#', 'processed_text',
       'topic_ids', 'subtopic', 'trump_pos_new', 'trump_neg_new',
       'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new',
       'Trump_flag', 'Biden_flag', 'Trump_Biden_flag'],
      dtype='object')

In [736]:
news_tweets['publication'] .value_counts()

The Washington Post    6999
CNN                    6108
Business Insider       3872
USA Today              3490
Breitbart News         3243
Fox News                872
Name: publication, dtype: int64

### Fav vs non-fav

In [756]:
user_resp.shape, user_resp.columns

((765796, 43),
 Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
        'Unnamed: 0.1', 'Unnamed: 0', 'tweet_id', 'conversation_id',
        'author_id', 'created_at', 'geo', 'lang', 'like_count', 'quote_count',
        'reply_count', 'retweet_count', 'source', 'text', 'anger', 'joy',
        'optimism', 'sadness', 'publication', 'topic_labels', 'theme',
        'pos_senti', 'neu_senti', 'neg_senti', 'date', 'week', 'flag',
        'Trump_flag', 'Biden_flag', 'Trump_Biden_flag', 'trump_pos',
        'trump_neg', 'trump_neu', 'biden_pos', 'biden_neg', 'biden_neu',
        'topics', 'topic_ids', 'subtopic'],
       dtype='object'))

In [ ]:
T_pos_men_CNN, T_neg_men_CNN = news_tweets[news_tweets['publication'] == 'CNN']

In [758]:
def get_pos_and_neg_mentions(df, pub, ent):

    pos_mentions = df[((df['publication'] == pub) & (df[ent + '_pos'] > df[ent + '_pos'].mean()))]
    neg_mentions = df[((df['publication'] == pub) & (df[ent + '_neg'] > df[ent + '_neg'].mean()))]
    
    return pos_mentions, neg_mentions

In [759]:
T_pos_men_CNN, T_neg_men_CNN = get_pos_and_neg_mentions(user_resp, 'CNN', 'trump')
B_pos_men_CNN, B_neg_men_CNN = get_pos_and_neg_mentions(user_resp, 'CNN', 'biden')

T_pos_men_TWP, T_neg_men_TWP = get_pos_and_neg_mentions(user_resp, 'The Washington Post', 'trump')
B_pos_men_TWP, B_neg_men_TWP = get_pos_and_neg_mentions(user_resp, 'The Washington Post', 'biden')

T_pos_men_UST, T_neg_men_UST = get_pos_and_neg_mentions(user_resp, 'USA Today', 'trump')
B_pos_men_UST, B_neg_men_UST = get_pos_and_neg_mentions(user_resp, 'USA Today', 'biden')

T_pos_men_BI, T_neg_men_BI = get_pos_and_neg_mentions(user_resp, 'Business Insider', 'trump')
B_pos_men_BI, B_neg_men_BI = get_pos_and_neg_mentions(user_resp, 'Business Insider', 'biden')

T_pos_men_FN, T_neg_men_FN = get_pos_and_neg_mentions(user_resp, 'Fox News', 'trump')
B_pos_men_FN, B_neg_men_FN = get_pos_and_neg_mentions(user_resp, 'Fox News', 'biden')

T_pos_men_BN, T_neg_men_BN = get_pos_and_neg_mentions(user_resp, 'Breitbart News', 'trump')
B_pos_men_BN, B_neg_men_BN = get_pos_and_neg_mentions(user_resp, 'Breitbart News', 'biden')

In [760]:
T_pos_men_FN.shape, T_neg_men_FN.shape, B_pos_men_FN.shape, B_neg_men_FN.shape

((13469, 43), (32956, 43), (31881, 43), (23450, 43))

In [761]:
T_pos_men_BN.shape, T_neg_men_BN.shape, B_pos_men_BN.shape, B_neg_men_BN.shape

((8532, 43), (24488, 43), (12638, 43), (14865, 43))

In [763]:
R_fav = pd.concat((T_pos_men_FN, T_pos_men_BN, B_neg_men_FN, B_neg_men_BN), axis = 0)
R_unfav = pd.concat((T_neg_men_FN, T_neg_men_BN, B_pos_men_FN, B_pos_men_BN), axis = 0)
print(R_fav.shape, R_unfav.shape)

L_fav = pd.concat((B_pos_men_CNN, B_pos_men_TWP, T_neg_men_CNN, T_neg_men_TWP), axis = 0)
L_unfav = pd.concat((T_pos_men_CNN, T_pos_men_TWP, B_neg_men_CNN, B_neg_men_TWP,), axis = 0)
L_fav.shape, L_unfav.shape

(60316, 43) (101963, 43)


((322635, 43), (167388, 43))

In [764]:
def get_pos_neg_mentions(df, entity):
    
    entity_mentions = df[df[entity.title() + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos'] > pos_mean))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg'] > neg_mean))]
    
    return pos_entity_mentions, neg_entity_mentions

def get_normalized_dist(dist):
    
    return (dist - dist.min()) / (dist.max() - dist.min())

def get_normalized_emotion_scores(df):
    
    df['anger_norm'] = get_normalized_dist(df.anger)
    df['optimism_norm'] = get_normalized_dist(df.optimism)
    df['sadness_norm'] = get_normalized_dist(df.sadness)
    df['joy_norm'] = get_normalized_dist(df.joy)
    
    return df

# def get_normalized_emotion_scores(df):
    
#     from sklearn.preprocessing import MinMaxScaler
    
#     scaler = MinMaxScaler()

#     df_ = df[['anger', 'optimism', 'sadness', 'joy']]
#     normalized_data = scaler.fit_transform(df_)
#     df_ = pd.DataFrame(normalized_data, columns = ['anger_norm', 'optimism_norm', 'sadness_norm', 'joy_norm'])
#     df = pd.concat((df, df_), axis = 1)

#     return df

In [699]:
L_user_resp = get_normalized_emotion_scores(L_user_resp)
R_user_resp = get_normalized_emotion_scores(R_user_resp)

In [700]:
# # df_L_T = L_user_resp[L_user_resp.Trump_flag == True]

# L_unfav1, L_fav1 = get_pos_neg_mentions(L_user_resp[L_user_resp.Trump_flag == True], 'trump')
# L_fav2, L_unfav2 = get_pos_neg_mentions(L_user_resp[L_user_resp.Biden_flag == True], 'biden')

# L_fav = pd.concat((L_fav1, L_fav2), axis = 0)
# L_unfav = pd.concat((L_unfav1, L_unfav2), axis = 0)

# # df_L_T = L_user_resp[L_user_resp.Trump_flag == True]

# R_fav1, R_unfav1 = get_pos_neg_mentions(R_user_resp[R_user_resp.Trump_flag == True], 'trump')
# R_unfav2, R_fav2 = get_pos_neg_mentions(R_user_resp[R_user_resp.Biden_flag == True], 'biden')

# R_fav = pd.concat((R_fav1, R_fav2), axis = 0)
# R_unfav = pd.concat((R_unfav1, R_unfav2), axis = 0)

In [765]:
# L_unfav.trump_pos.mean(), L_fav.trump_pos.mean(), L_unfav.biden_pos.mean(), L_fav.biden_pos.mean()
# R_unfav.trump_pos.mean(), R_fav.trump_pos.mean(), R_unfav.biden_pos.mean(), R_fav.biden_pos.mean()

L_fav.shape, L_unfav.shape, R_fav.shape, R_unfav.shape

((322635, 43), (167388, 43), (60316, 43), (101963, 43))

In [767]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
L_fav[L_fav.anger.isna() == True][emotions].head()

,anger,joy,optimism,sadness


In [704]:
emotions = ['anger_norm', 'joy_norm', 'optimism_norm', 'sadness_norm']
L_fav[L_fav.anger_norm.isna() == True][emotions].head()

,anger_norm,joy_norm,optimism_norm,sadness_norm


In [753]:
R_fav.columns

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0',
       'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'Unnamed: 0.1.1.1', 'tweet_id', 'conversation_id', 'author_id',
       'created_at', 'geo', 'lang', 'like_count', 'quote_count', 'reply_count',
       'retweet_count', 'source', 'text', 'display_name', 'topics',
       'topic_labels', 'theme', 'pos_senti', 'neu_senti', 'neg_senti',
       'publication', 'date', 'week', 'polIncl', 'hashtag', 'clean_text',
       'proc_text', 'Segmented#', 'processed_text', 'topic_ids', 'subtopic',
       'trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new',
       'biden_neg_new', 'biden_neu_new', 'Trump_flag', 'Biden_flag',
       'Trump_Biden_flag', 'topic_id'],
      dtype='object')

### Overall Fav-vs-UnFav

In [710]:
# def get_significance_stats(dist1, dist2, pub, entity, emotion):
    
#     print(dist1.shape, dist2.shape)

#     emot1_shapiro = stats.shapiro(dist1)
#     emot2_shapiro = stats.shapiro(dist2)
#     emot1_shapiro = (emot1_shapiro.statistic, emot1_shapiro.pvalue)
#     emot2_shapiro = (emot2_shapiro.statistic, emot2_shapiro.pvalue)
#     H, p, dof, es = get_KW_stat(dist1, dist2)
#     # t, p , dof = welch_ttest(emot1, emot2)
#     res = [entity, pub, emotion, dist1.shape[0], dist2.shape[0], emot1_shapiro[0], emot1_shapiro[1], emot2_shapiro[0], emot2_shapiro[1], p, H, dof, es]
#     # print(res)
#     return res

In [772]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
# emotions = ['anger_norm', 'joy_norm', 'optimism_norm', 'sadness_norm']
sstat = []

for emot in emotions:
    
    for i, ur in enumerate([(R_fav, R_unfav),(L_fav, L_unfav)]):

        data1 = ur[0]
        data2 = ur[1]
        # print(data1.shape, data2.shape)
        dist1 = data1[emot].dropna()
        dist2 = data2[emot].dropna()
        
        # print(dist1.shape, dist2.shape)
        
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):
            
            if(i%2 == 0):
                res = get_significance_stats(dist1, dist2, 'R', None, emot)
            else:
                res = get_significance_stats(dist1, dist2, 'L', None, emot)
            # print(res)
            sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(5)
stats_df

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,None,R,anger,60316,101963,0.80430,0.0,0.80826,0.0,0.06053,3.52273,1,0.00002
1,None,L,anger,322635,167388,0.81889,0.0,0.81798,0.0,0.00000,103.60633,1,0.00021
2,None,R,joy,60316,101963,0.51218,0.0,0.53337,0.0,0.00279,8.94162,1,0.00006
3,None,L,joy,322635,167388,0.53487,0.0,0.51368,0.0,0.00000,174.75754,1,0.00036
4,None,R,optimism,60316,101963,0.59736,0.0,0.60487,0.0,0.00664,7.36758,1,0.00005
5,None,L,optimism,322635,167388,0.63182,0.0,0.62478,0.0,0.00000,27.29364,1,0.00006
6,None,R,sadness,60316,101963,0.60943,0.0,0.60795,0.0,0.00000,49.96613,1,0.00031
7,None,L,sadness,322635,167388,0.61693,0.0,0.62556,0.0,0.00000,44.47976,1,0.00009


In [775]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
# emotions = ['anger_norm', 'joy_norm', 'optimism_norm', 'sadness_norm']
sstat = []
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))

for emot in emotions:
    
    for top in topics:

        for i, ur in enumerate([(R_fav, R_unfav),(L_fav, L_unfav)]):
        
            data1 = ur[0]
            data2 = ur[1]
            dist1 = data1[data1.topics == top][emot].dropna()
            dist2 = data2[data2.topics == top][emot].dropna()

            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                if(i%2 == 0):
                    res = get_significance_stats(dist1, dist2, 'R', top, emot)
                else:
                    res = get_significance_stats(dist1, dist2, 'L', top, emot)
                # print(res)
                sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
# stats_df

In [776]:
stats_df[stats_df['Entity'] == 'R']

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,capitol,R,anger,626,3205,0.705,0.0,0.729,0.000,0.040,4.225,1,0.001
2,republican,R,anger,653,157,0.822,0.0,0.817,0.000,0.603,0.271,1,0.000
4,economy,R,anger,2692,310,0.808,0.0,0.856,0.000,0.005,8.034,1,0.003
6,fact-check,R,anger,1388,129,0.747,0.0,0.830,0.000,0.054,3.727,1,0.002
8,security,R,anger,174,2255,0.829,0.0,0.776,0.000,0.001,11.622,1,0.005
10,others,R,anger,21597,16996,0.807,0.0,0.819,0.000,0.000,35.613,1,0.001
12,media,R,anger,187,4040,0.803,0.0,0.838,0.000,0.003,8.812,1,0.002
14,democarts,R,anger,3239,16115,0.810,0.0,0.807,0.000,0.038,4.320,1,0.000
16,america,R,anger,1147,18,0.819,0.0,0.844,0.007,0.331,0.947,1,0.001
18,election,R,anger,7625,23774,0.790,0.0,0.803,0.000,0.000,13.674,1,0.000


In [666]:
stats_df

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,capitol,R,anger,626,3205,0.705,0.0,0.729,0.000,0.040,4.225,1,0.001
1,capitol,L,anger,9317,3512,0.788,0.0,0.762,0.000,0.000,29.902,1,0.002
2,republican,R,anger,653,564,0.822,0.0,0.850,0.000,0.003,8.546,1,0.007
3,republican,L,anger,6620,1987,0.810,0.0,0.827,0.000,0.756,0.096,1,0.000
4,economy,R,anger,2692,310,0.808,0.0,0.856,0.000,0.005,8.034,1,0.003
5,economy,L,anger,5143,2229,0.827,0.0,0.863,0.000,0.000,19.914,1,0.003
6,fact-check,R,anger,1388,129,0.747,0.0,0.830,0.000,0.054,3.727,1,0.002
7,fact-check,L,anger,7581,5176,0.793,0.0,0.860,0.000,0.000,180.869,1,0.014
8,security,R,anger,174,2255,0.829,0.0,0.776,0.000,0.001,11.622,1,0.005
9,security,L,anger,13872,7691,0.809,0.0,0.796,0.000,0.000,41.147,1,0.002


In [644]:
stats_df.sort_values(by='KW-p', ascending=False)

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
55,america,L,joy,3556,4251,0.538,0.0,0.494,0.000,0.997,0.000,1,0.000
36,pres_debate,R,joy,4142,4594,0.485,0.0,0.483,0.000,0.964,0.002,1,0.000
46,international,R,joy,47,220,0.563,0.0,0.529,0.000,0.943,0.005,1,0.000
63,covid,L,joy,51439,24415,0.501,0.0,0.499,0.000,0.937,0.006,1,0.000
82,international,R,optimism,47,220,0.567,0.0,0.595,0.000,0.915,0.011,1,0.000
47,international,L,joy,3541,6148,0.508,0.0,0.525,0.000,0.907,0.014,1,0.000
32,immigration,R,anger,350,188,0.837,0.0,0.822,0.000,0.904,0.015,1,0.000
119,international,L,sadness,3541,6148,0.625,0.0,0.630,0.000,0.900,0.016,1,0.000
118,international,R,sadness,47,220,0.753,0.0,0.718,0.000,0.899,0.016,1,0.000
140,immigration,R,sadness,350,188,0.773,0.0,0.712,0.000,0.892,0.018,1,0.000


In [ ]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
sstat = []
for emot in emotions:
    
    for ur in [L_user_resp, R_user_resp]:

        dataFav = ur[ur[ent.title() + '_flag'] == True]
        dataR = R_user_resp[R_user_resp[ent.title() + '_flag'] == True]

    # for top in topics:

        dist1 = dataL[dataL.topics == top][emot].dropna()
        dist2 = dataR[dataR.topics == top][emot].dropna()

        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ent, top, emot)
            # print(res)
            sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

In [ ]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
sstat = []
for emot in emotions:

    dataFav = L_user_resp[L_user_resp[ent.title() + '_flag'] == True]
    dataR = R_user_resp[R_user_resp[ent.title() + '_flag'] == True]

    for top in topics:

        dist1 = dataL[dataL.topics == top][emot].dropna()
        dist2 = dataR[dataR.topics == top][emot].dropna()

        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ent, top, emot)
            # print(res)
            sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

### Left vs Right (emotion distributions) based on topics

In [587]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
sstat = []
for emot in emotions:

    for top in topics:

        dist1 = L_user_resp[L_user_resp.topics == top][emot].dropna()
        dist2 = R_user_resp[R_user_resp.topics == top][emot].dropna()

        # print(dist2.shape, dist1.shape)

        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ent, top, emot)
            # print(res)
            sstat.append(res)

        
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,pres_debate,trump,anger,10034,5324,0.789,0.0,0.783,0.0,0.004,8.099,1,0.001
1,economy,trump,anger,8096,4207,0.833,0.0,0.815,0.0,0.000,22.668,1,0.002
2,others,trump,anger,150168,45012,0.829,0.0,0.809,0.0,0.000,464.044,1,0.002
3,fact-check,trump,anger,14001,1460,0.816,0.0,0.755,0.0,0.000,36.659,1,0.002
4,blm,trump,anger,14838,5523,0.783,0.0,0.802,0.0,0.000,24.183,1,0.001
5,international,trump,anger,8611,719,0.822,0.0,0.842,0.0,0.000,13.821,1,0.001
6,supreme_court,trump,anger,25720,5330,0.828,0.0,0.794,0.0,0.000,171.917,1,0.006
7,media,trump,anger,16445,7158,0.788,0.0,0.827,0.0,0.000,136.719,1,0.006
8,democarts,trump,anger,27540,21555,0.827,0.0,0.812,0.0,0.000,33.533,1,0.001
9,america,trump,anger,6516,1728,0.821,0.0,0.833,0.0,0.931,0.008,1,0.000


In [347]:
def get_pos_neg_mentions(df, entity):
    
    entity_mentions = df[df[entity.title() + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos'] > pos_mean))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg'] > neg_mean))]
    
    return pos_entity_mentions, neg_entity_mentions

In [348]:
T_pos_mentions_R,  T_neg_mentions_R = get_pos_neg_mentions(R_user_resp, "trump")
B_pos_mentions_R,  B_neg_mentions_R = get_pos_neg_mentions(R_user_resp, "biden")

T_pos_mentions_L,  T_neg_mentions_L = get_pos_neg_mentions(L_user_resp, "trump")
B_pos_mentions_L,  B_neg_mentions_L = get_pos_neg_mentions(L_user_resp, "biden")

In [351]:
# T_pos_mentions_R.shape, T_neg_mentions_R.shape, B_pos_mentions_R.shape, B_neg_mentions_R.shape, T_pos_mentions_L.shape, T_neg_mentions_L.shape, B_pos_mentions_L.shape, B_neg_mentions_L.shape

In [371]:
def get_mean_emotions_per_topic_for_pos_neg_entity_mentions(df1, df2, cols):
    
    return pd.concat((df1.groupby('topics').mean()[cols].rename(columns = {'anger': 'anger_P', 'joy': 'joy_P', 'optimism': 'optimism_P', 'sadness': 'sadness_P'}), 
           df2.groupby('topics').mean()[cols].rename(columns = {'anger': 'anger_N', 'joy': 'joy_N', 'optimism': 'optimism_N', 'sadness': 'sadness_N'})), axis = 1)

R_fav = pd.concat((T_pos_mentions_R, B_neg_mentions_R), axis = 0)
R_unfav = pd.concat((T_neg_mentions_R, B_pos_mentions_R), axis = 0)
R_fav.shape, R_unfav.shape

L_fav = pd.concat((T_neg_mentions_L, B_pos_mentions_L), axis = 0)
L_unfav = pd.concat((T_pos_mentions_L, B_neg_mentions_L), axis = 0)
L_fav.shape, L_unfav.shape

# get_mean_emotions_per_topic_for_pos_neg_entity_mentions(L_fav, L_unfav, cols).to_csv('results/user_resp_analysis/L_fav_vs_unfav_topic_wise.csv')

((320428, 43), (169575, 43))

In [409]:
import warnings
warnings.filterwarnings('ignore')

### Comparing emotion distributions in user response between positive and negative mentions of an entity 

In [397]:
def get_pos_neg_mentions(df, entity):
    
    entity_mentions = df[df[entity.title() + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos'] > pos_mean))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg'] > neg_mean))]
    
    return pos_entity_mentions, neg_entity_mentions

In [398]:
T_pos_mentions_R,  T_neg_mentions_R = get_pos_neg_mentions(R_user_resp, "trump")
B_pos_mentions_R,  B_neg_mentions_R = get_pos_neg_mentions(R_user_resp, "biden")

T_pos_mentions_L,  T_neg_mentions_L = get_pos_neg_mentions(L_user_resp, "trump")
B_pos_mentions_L,  B_neg_mentions_L = get_pos_neg_mentions(L_user_resp, "biden")

In [440]:
emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(L_user_resp.topics).intersection(set(R_user_resp.topics))
sstat = []

def get_stats_df(df1, df2, ent):
    
    for emot in emotions:

        # for top in topics:

        dist1 = T_pos_mentions_R[emot].dropna()
        dist2 = T_neg_mentions_R[emot].dropna()
            
            # print(set(df1.topics))
            
            # sys.exit()

            # if(top in set(df1.topics) and top in set(df2.topics)):

            # dist1 = df1[df1.topics == top][emot].dropna()
            # dist2 = df2[df2.topics == top][emot].dropna()

        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ent, top, emot)
            # print(res)
            sstat.append(res)
    
    return sstat
        
df1 = B_pos_mentions_L
df2 = B_neg_mentions_L

sstat = get_stats_df(df1, df2, ent = 'Biden')
stats_df = pd.DataFrame(sstat, columns = ['Topic', 'Entity', 'emotion', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'shapiro_t_dist1', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)

In [441]:
stats_df.sort_values(by='ES', ascending=False)

,Topic,Entity,emotion,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist2,shapiro_t_dist1,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
2,pres_debate,Biden,optimism,21919,57444,0.623,0.0,0.588,0.0,0.000,252.625,1,0.003
0,pres_debate,Biden,anger,21919,57444,0.807,0.0,0.789,0.0,0.000,159.892,1,0.002
3,pres_debate,Biden,sadness,21919,57444,0.607,0.0,0.586,0.0,0.000,105.184,1,0.001
1,pres_debate,Biden,joy,21919,57444,0.508,0.0,0.504,0.0,0.001,11.206,1,0.000
